# Finding Anomolies

## Setup

### Import Data

In [52]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import warnings
import geopy.distance

warnings.filterwarnings('ignore')

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

storage = "/Volumes/easystore/Drones/"

flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

In [53]:
geocodio_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/geocodio/all-flights-manifest_geocodio.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

In [54]:
flight_details["geoid"] = flight_details["geoid"].astype(str)
flight_details["len"] = flight_details["geoid"].apply(lambda x: len(x))
flight_details.loc[flight_details["len"] == 14, "geoid"] = "0" + flight_details["geoid"]


In [55]:
geocodio_flights["date"] = pd.to_datetime(geocodio_flights["date"]).dt.date


### Launch Sites
- BayView Hospital = 60730131022000
- CVPD = 60730123021013
- SharpChula Medical Center = 60730133273000
- SWestern College = 60730134151000


## Functions

In [56]:
def compile_anomolies(anomolies):
    anomolies = anomolies[anomolies['id']!=False].copy()
    
    anomolies = pd.merge(anomolies,geocodio_flights,how='left',on='id')
    anomolies['fn'] = anomolies['id'].apply(lambda x: f"/Volumes/easystore/Drones/flights/kml/{x}.kml")

    compiled_flights = []
    for _, row in anomolies.iterrows():
        d = gpd.read_file(row['fn'], driver='KML')


        d['id'] = row['id']
        d['incident_type'] = row['type_y']
        d['incident_id'] = row['incident_id']
        d['address_map'] = row['address_map']
        d['date'] = row['date']
        d['time'] = row['time_s']
        d['lingering_block'] = row['lingering_block']
        d['seconds_lingering'] = row['seconds_lingering']
        d['destination_block'] = str(row['destination_block'])
        d['distance'] = str(row['distance'])
        d['latitude'] = row['Latitude']
        d['longitude'] = row['Longitude']
        d['lat_map'] = row['lat_map']
        d['lon_map'] = row['lon_map']
        d['starting_block'] = str(row['starting_block'])

        compiled_flights.append(d)
    return pd.concat(compiled_flights,ignore_index=True)

In [57]:
def get_unique_seconds_for_block(flight):

    times = pd.to_datetime(flight["sequence"])
    grouped_by_block_and_second = flight.groupby(
    ["id","geoid","block_group","tract", times.dt.hour, times.dt.minute, times.dt.second]).count()
    grouped_by_block_and_second.index.names = ["id","geoid","block_group","tract", "hour", "minute", "second"]
    grouped_by_block_and_second = grouped_by_block_and_second.reset_index()[
    ["id","geoid","block_group","tract", "hour", "minute", "second", "type"]
    ]
    grouped_by_block_and_second.columns = ["id","geoid","block_group","tract", "hour", "minute", "second", "count"]
    
    unique_seconds_in_block = grouped_by_block_and_second.groupby(["geoid","block_group","tract"]).count()
    unique_seconds_in_block = unique_seconds_in_block.reset_index()[["geoid","block_group","tract", "hour"]]
    unique_seconds_in_block.columns = ["GEOID20","block_group","tract","seconds"]
    return unique_seconds_in_block

## Searching

In [58]:

LINGER = 300
def filter_one(index,details=False):
   
    if details == True:
        flight_id = index
        flight = flight_details[flight_details['id']==flight_id]
    else:
        flight_id = flight_ids.iloc[index]
        flight = flight_details[flight_details['id']==flight_id]
    try:
#       get info from the map for corresponding flight
        manifest_details = geocodio_flights[geocodio_flights['id']==flight_id]
        destination_block = manifest_details.iloc[0][['Full FIPS (block)','Census Tract Code', 'Census Block Group','Latitude', 'Longitude']]
# convert to int because im paranoid
        destination_block['Census Tract Code'] = int(destination_block['Census Tract Code'])
        destination_block['Full FIPS (block)'] = int(destination_block['Full FIPS (block)'])
# finding starting block of flight
        starting_block = int(flight.iloc[0]['geoid'])
# Calculate Number of Seconds Drone Spent in Every Block
        unique_seconds_in_block = get_unique_seconds_for_block(flight)
        unique_seconds_in_block['GEOID20'].astype(int)
#       Remove Starting and Destination Block from analysis because we want to find all flights that spent time over somewhere other than the stated destination.        
        unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=starting_block].copy()
        unique_seconds_in_block =  unique_seconds_in_block[unique_seconds_in_block['GEOID20']!=destination_block['Full FIPS (block)']].copy()
# Of the remaining blocks, identify the ones where drones spent more than 5 min.
        lingering_blocks = unique_seconds_in_block[unique_seconds_in_block['seconds']>LINGER]
        # lingering_blocks=lingering_blocks[~lingering_blocks['GEOID20'].isin(blacklist)]
        anomoly = {"id":False,"type":False,'starting_block':starting_block,"destination_tract":destination_block['Census Tract Code'],"lingering_block":False,"seconds_lingering":False}
#         consider increasing
#       If there is more than one block where a drone spent 5 minutes that isn't the starting or destinaion block calculate the distance from the lingering block to the stated destination.  if its more than a mile, flag it as an anomoly.
        if lingering_blocks.shape[0]>0:
            lingering_blocks_far_from_dst = []
            for _,row in lingering_blocks.iterrows():
                lingering_lat,lingering_lng = flight[flight['geoid'] == row['GEOID20']][['latitude','longitude']].values[0]
                distance = geopy.distance.geodesic((lingering_lat,lingering_lng), (destination_block['Latitude'], destination_block['Longitude'])).miles
                if distance > 1:
                    anomoly ={"id":flight_id,"type":flight['type'].drop_duplicates().values[0],'starting_block':starting_block,"destination_block":int(destination_block['Full FIPS (block)']),"distance":distance,"lingering_block":row['GEOID20'],"seconds_lingering":row['seconds']}
        if details == True:
            return [anomoly,unique_seconds_in_block,starting_block,destination_block]
        else:
            return anomoly
    except:
        print("error")
        return {"id":False,"type":False,'starting_block':False,"destination_tract":False,"lingering_block":False,"seconds_lingering":False}
    


In [ ]:
anomoly,unique_seconds_in_block,starting_block,destination_block = filter_one('fa1a59574dd96d6262f17fe9fd20ca17',True)

In [ ]:
# def calc_dist(flight_id):
#     flight = geocodio_flights[geocodio_flights['id']==flight_id]
#     dist = geopy.distance.geodesic((flight['lat_map'].values[0],flight['lon_map'].values[0]), (flight['Latitude'].values[0], flight['Longitude'].values[0])).miles
#     return dist

# geocodio_flights['map-delta'] = geocodio_flights['id'].apply(lambda x: calc_dist(x))
# stable_flights = geocodio_flights[geocodio_flights['map-delta']<=.16]


In [ ]:
# stable_flights

In [59]:
# flight_ids = stable_flights.sample(1000)['id'].drop_duplicates()
flight_ids = flight_details[['id']].sample(frac=1).drop_duplicates()
flight_ids = flight_ids[flight_ids['id'].isin(geocodio_flights['id'])]['id']

with Pool(10) as pool:

    lingering_nd = list(
        tqdm(pool.imap(filter_one, range(0, flight_ids.shape[0])), total=flight_ids.shape[0])
    )
    lingering_nd = pd.DataFrame(lingering_nd)

  0%|          | 0/11071 [00:00<?, ?it/s]

error


In [60]:
lingering_anomolies = compile_anomolies(lingering_nd)
lingering_anomolies = gpd.GeoDataFrame(lingering_anomolies)
lingering_anomolies['incident_type'] = lingering_anomolies['incident_type'].astype(str)
lingering_anomolies['destination_block']=lingering_anomolies['destination_block'].astype(float).astype(int)
lingering_anomolies['starting_block']=lingering_anomolies['starting_block'].astype(float).astype(int)
lingering_anomolies['lingering_block']=lingering_anomolies['lingering_block'].astype(float).astype(int)

lingering_anomolies['id'].drop_duplicates().shape[0]


827

In [61]:
lingering_anomolies[['id','geometry','incident_type','distance','lingering_block','destination_block','longitude','latitude','address_map','starting_block']].to_file("../../data/anomolies/lingering-one-mile-from-destination.geojson", driver='GeoJSON')
